<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/sample_1000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Saving a 1000 dev set for manual coding



In [0]:
DATASET_NAME = "status_posts"
MODEL_NAME = "status_posts"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 20
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [24]:
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

In [25]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

Updated property [core/project].


In [26]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
from google.colab import files
import io 

In [0]:
# Get 1000 random indices for the dev set.
rand_index = np.random.randint(0,len(dev_df),1000)
rand_index=rand_index.tolist()

# Create a new dataframe for manual coding of 1000 random respones.
sample_1000 = dev_df.iloc[rand_index]
sample_1000 = sample_1000[['original_idx','response_text']]


In [0]:
# Download the new dataframe to local machine.

sample_1000.to_csv('sample_1000.csv',encoding='utf-8-sig', header=True, index=False)
files.download('sample_1000.csv') 


## Analysis of manually coded data

In [29]:
# Get manually coded csv files.

uploaded = files.upload()
sample_1000 = pd.read_csv(io.BytesIO(uploaded['sample_1000.csv']))

Saving sample_1000.csv to sample_1000 (1).csv


In [30]:
sample_1000.LP_Gender.value_counts()

M    722
W    278
Name: LP_Gender, dtype: int64

In [31]:
sample_1000.ER_Gender.value_counts()

M    792
W    208
Name: ER_Gender, dtype: int64

In [0]:
dev_sample_1000 = pd.merge(sample_1000,dev_df[['original_idx','op_gender','post_id','responder_id','op_name','op_category']], on='original_idx', how = 'left')

In [0]:
LP_correct = dev_sample_1000[dev_sample_1000['LP_Gender']==dev_sample_1000['op_gender']]
LP_incorrect = dev_sample_1000[dev_sample_1000['LP_Gender']!=dev_sample_1000['op_gender']]
ER_correct = dev_sample_1000[dev_sample_1000['ER_Gender']==dev_sample_1000['op_gender']]
ER_incorrect = dev_sample_1000[dev_sample_1000['ER_Gender']!=dev_sample_1000['op_gender']]

In [34]:
gender_counts_LP = LP_correct.LP_Gender.value_counts()
gender_counts_ER = ER_correct.ER_Gender.value_counts()

print('Lorrel got {} correct: {} M, {} W'.format(len(LP_correct),gender_counts_LP[0],gender_counts_LP[1]))
print('Emily got {} correct: {} M, {} W'.format(len(ER_correct),gender_counts_ER[0],gender_counts_ER[1]))


Lorrel got 746 correct: 664 M, 82 W
Emily got 780 correct: 716 M, 64 W


In [0]:
both_correct = LP_correct[LP_correct['LP_Gender']==LP_correct['ER_Gender']]

In [38]:
both_correct.head()

,original_idx,response_text,LP_Gender,ER_Gender,op_gender,post_id,responder_id,op_name,op_category
0,10310792,show your taxes.,M,M,M,382223,Sylvia,Tim Kaine,Congress_Democratic
2,7592407,i doubt he has heading for the us. I had suspect more likely organizing in brazil.,M,M,M,291510,Evad,Ryan Zinke,Congress_Republican
3,7994949,government out of control!!! http://www.facebook.com/home.php?ref=home&__a=1#/group.php?gid=880...,M,M,M,296909,Tom,John McCain,Congress_Republican
4,5274206,saving america......i used to work for the army in an interservice training and support organiza...,M,M,M,161376,Stephen,Marco Rubio,Congress_Republican
6,10982900,"my response to my son's friend who unfriended him for voting for trump: ""do not me mad that you...",W,W,W,389366,Gloria,Kirsten Gillibrand,Congress_Democratic


In [39]:
len(both_correct)

642

In [41]:
both_correct.LP_Gender.value_counts()

M    593
W     49
Name: LP_Gender, dtype: int64

In [40]:
both_incorrect = LP_incorrect[LP_incorrect['LP_Gender']==LP_incorrect['ER_Gender']]
len(both_incorrect)

116

In [42]:
both_incorrect.LP_Gender.value_counts()

W    73
M    43
Name: LP_Gender, dtype: int64

In [43]:
both_incorrect.head(10)

,original_idx,response_text,LP_Gender,ER_Gender,op_gender,post_id,responder_id,op_name,op_category
5,5358947,hope you win!,W,W,M,161811,Corry,Marco Rubio,Congress_Republican
18,10876594,"hmmm, can someone educate me here, how is passing a law going to protect people in another count...",M,M,W,385147,Kirsten,Anna Eshoo,Congress_Democratic
20,5379599,catherine. you just proved her point. she doesnt want her pay to be based on the test... neither...,W,W,M,162099,Debra-Marie,Marco Rubio,Congress_Republican
26,3242044,i have unsubscribed from your mailing list. your decision not to support the iran deal was a rea...,W,W,M,99377,Donna,Bob Menendez,Congress_Democratic
41,11294395,proud to have served as a marine from 1960 - 1964. was on a ship off the south coast of cuba in ...,M,M,W,400418,Ron,Cathy McMorris Rodgers,Congress_Republican
51,13460854,omg! we have 2 more years of this!,W,W,M,517043,Nancy,Steve Scalise,Congress_Republican
53,7830011,if anyone is able to help we greatly appreciate it. even just sharing our story might help us to...,W,W,M,296303,Jason,John McCain,Congress_Republican
54,7856037,what sandy said. ^^^,W,W,M,296338,Jodi,John McCain,Congress_Republican
57,10978180,damn your a fucking idiot....,M,M,W,389348,Curtis,Kirsten Gillibrand,Congress_Democratic
60,4800399,vote last tuesday and you got my vote and my husband,W,W,M,160091,Alice,Marco Rubio,Congress_Republican


In [44]:
# Lorrel correct, Emily incorrect.
L_corr_E_incorr = LP_correct[LP_correct['LP_Gender']!=LP_correct['ER_Gender']]
L_corr_E_incorr.LP_Gender.value_counts()

M    71
W    33
Name: LP_Gender, dtype: int64

In [46]:
L_corr_E_incorr.head(10)

,original_idx,response_text,LP_Gender,ER_Gender,op_gender,post_id,responder_id,op_name,op_category
16,10883069,[[sticker]],M,W,M,385678,Shigeko,Mark Takano,Congress_Democratic
28,11066364,senator you have earned and deserve our support. congradulations.,W,M,W,390145,Michael,Kirsten Gillibrand,Congress_Democratic
34,2539484,so if a congressional member rapes your kids will the oce be able to call the cops? not anymor...,M,W,M,63420,Jeremy,Paul Ryan,Congress_Republican
50,11258691,food not bombs. gardens not lawns.,W,M,W,399674,DeeDee,Cathy McMorris Rodgers,Congress_Republican
64,2589757,endorse trump! you sound like clinton and obama this the radicial islam cannot say it thing. ...,M,W,M,63460,Mike,Paul Ryan,Congress_Republican
68,11017521,I have never understood why firefighters and police officers were sent to clean this rubble. th...,W,M,W,389723,Eileen,Kirsten Gillibrand,Congress_Democratic
71,10047776,"yes, rep king you did see and warned everyone.... unfortunately way to many did not want to list...",M,W,M,365841,Thelma,Steve King,Congress_Republican
85,10038686,"congressman king, i am a long-time hospice and palliative care social worker. i have worked in c...",M,W,M,365393,Meredith,Steve King,Congress_Republican
90,11272166,"i usually do not agree with you representative mcmorris rodgers, but on this one I am completely...",W,M,W,399770,Mark,Cathy McMorris Rodgers,Congress_Republican
96,2137230,nanny state fascist. no vote for you.,M,W,M,36398,Martin,Sherrod Brown,Congress_Democratic


In [45]:
# Emily correct, Lorrel incorrect.
E_corr_L_incorr = ER_correct[ER_correct['ER_Gender']!=ER_correct['LP_Gender']]
E_corr_L_incorr.ER_Gender.value_counts()

M    123
W     15
Name: ER_Gender, dtype: int64

In [48]:
E_corr_L_incorr.head(10)

,original_idx,response_text,LP_Gender,ER_Gender,op_gender,post_id,responder_id,op_name,op_category
1,11000856,"I will remember this, come your re-election. i will not vote for you again.",M,W,W,389558,Rise,Kirsten Gillibrand,Congress_Democratic
11,7585262,so sorry this happened. prayers for this family.,W,M,M,291266,Karen,Ryan Zinke,Congress_Republican
13,7714188,"thank you for speaking out, senator.",W,M,M,296100,Sarah,John McCain,Congress_Republican
25,2172894,i wish i could vote for you but i do support your work and your ideals!,W,M,M,36883,Renee,Sherrod Brown,Congress_Democratic
52,2135200,my faith tells me that we all need and deserve second chances. i urge you to support reauthoriza...,W,M,M,36365,Eric,Sherrod Brown,Congress_Democratic
58,10129623,thankful for you.,W,M,M,370568,Karen,Richard Shelby,Congress_Republican
76,2192417,wow in one month the unemployment went down to 9% way to go gop,W,M,M,37895,Jeffrey,Sherrod Brown,Congress_Democratic
77,2535842,the quinnipiac university poll shows that just 16 percent of americans want congress to repeal a...,W,M,M,63416,Carla,Paul Ryan,Congress_Republican
91,4950654,i am sick and tired of the college people getting a bad rap when it comes to american politics. ...,W,M,M,160273,Mark,Marco Rubio,Congress_Republican
93,5329914,wow guys...............it has not flip flopping to admit you were wrong. politicians can make m...,W,M,M,161552,Erica,Marco Rubio,Congress_Republican
